In [ ]:
from google.colab import drive
drive.mount('/content/drive')

BASE = '/content/drive/MyDrive/MSML612'

RUNS = f'{BASE}/runs'
DATA_ROOT = f"{BASE}/brats_final_split"

LOCAL = "/content/MSML612_work"
LOCAL_IMG_ROOT = f"{LOCAL}/brats_final_split_flair"

import os
for d in [BASE, RUNS, LOCAL, LOCAL_IMG_ROOT]:
    os.makedirs(d, exist_ok=True)

print("BASE:", BASE)
print("DATA_ROOT:", DATA_ROOT)
!find "$DATA_ROOT" -maxdepth 3 -type d -printf '%P\n' | head -n 20

Mounted at /content/drive
BASE: /content/drive/MyDrive/MSML612
DATA_ROOT: /content/drive/MyDrive/MSML612/brats_final_split


In [ ]:
import os, shutil

def copy_flair_split(split):
    src = os.path.join(DATA_ROOT, split, "flair")
    dst = os.path.join(LOCAL_IMG_ROOT, split)
    if os.path.exists(dst) and os.listdir(dst):
        print(f"{split} already copied to local: {dst}")
        return
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    print(f"Copying {split} flair slices from {src} -> {dst} ...")
    shutil.copytree(src, dst)
    print("Done:", split)

for split in ["train", "val", "test"]:
    copy_flair_split(split)

def count_pngs(folder):
    total = 0
    for root, _, files in os.walk(folder):
        total += sum(1 for f in files if f.lower().endswith(".png"))
    return total

for split in ["train", "val", "test"]:
    sp = os.path.join(LOCAL_IMG_ROOT, split)
    print(split, "pngs:", count_pngs(sp))

train already copied to local: /content/MSML612_work/brats_final_split_flair/train
val already copied to local: /content/MSML612_work/brats_final_split_flair/val
test already copied to local: /content/MSML612_work/brats_final_split_flair/test
train pngs: 9671
val pngs: 2072
test pngs: 2073


In [ ]:
!pip uninstall -y diffusers huggingface_hub transformers accelerate -q
!pip install -q "diffusers==0.29.2" "huggingface_hub==0.23.4" "transformers==4.41.1" "accelerate==0.30.1" "safetensors>=0.4.2"

# %cd "$BASE"
# !rm -rf Seg_diffusion
# !git clone https://github.com/PranavShashidhara/Seg_diffusion.git

%cd "$BASE/Seg_diffusion"
!pip install -q -r requirements.txt

%cd /content
!test -f "{BASE}/Seg_diffusion/main.py" && echo "main.py found" || echo "main.py missing"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.0 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.23.4 which is incompatible.
gradio 5.50.0 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.23.4 which is incompatible.
/content/drive/MyDrive/MSML612/Seg_diffusion
/content
main.py found


In [ ]:
IMG_SIZE = 256
IMG_DIR = LOCAL_IMG_ROOT

OUTPUT = f"{RUNS}/baseline_uncond_ddim"
os.makedirs(OUTPUT, exist_ok=True)

print("Using IMG_DIR:", IMG_DIR)
print("Saving logs to:", OUTPUT)

!CUDA_VISIBLE_DEVICES=0 python3 "{BASE}/Seg_diffusion/main.py" \
  --mode train \
  --model_type DDIM \
  --img_size {IMG_SIZE} \
  --num_img_channels 1 \
  --img_dir "{IMG_DIR}" \
  --train_batch_size 24 \
  --num_epochs 10 \
  --eval_batch_size 24 \
  --eval_sample_size 32 \
  2>&1 | tee "{OUTPUT}/train.log"


Using IMG_DIR: /content/MSML612_work/brats_final_split_flair
Saving logs to: /content/drive/MyDrive/MSML612/runs/baseline_uncond_ddim
2025-11-28 17:56:02.746342: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-28 17:56:02.764041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764352562.784922   49414 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764352562.791576   49414 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:

In [ ]:
import shutil
import os

src = "/content/ddim-breast_mri-256"
dst = "/content/drive/MyDrive/MSML612/ddim-breast_mri-256"

os.makedirs(os.path.dirname(dst), exist_ok=True)

print("Copying model to Drive...")
shutil.copytree(src, dst, dirs_exist_ok=True)
print("Done! Model stored at:", dst)

Copying model to Drive...
Done! Model stored at: /content/drive/MyDrive/MSML612/ddim-breast_mri-256


In [ ]:
import os
from diffusers import DiffusionPipeline
import torch

MODEL_DIR = f"{BASE}/ddim-breast_mri-256"

MASK_DIR = os.path.join(DATA_ROOT, "test", "mask")
SAVE_DIR = os.path.join(MODEL_DIR, "samples_from_test_masks")
os.makedirs(SAVE_DIR, exist_ok=True)

all_masks = sorted(
    f for f in os.listdir(MASK_DIR)
    if f.lower().endswith((".png", ".jpg", ".jpeg"))
)
mask_files = all_masks[:100]
print(f"Found {len(mask_files)} mask files, will generate {len(mask_files)} images.")

print("Loading pipeline from:", MODEL_DIR)
pipe = DiffusionPipeline.from_pretrained(
    MODEL_DIR,
    torch_dtype=torch.float16,
).to("cuda")

generator = torch.Generator(device="cuda").manual_seed(42)

batch_size = 8
for start in range(0, len(mask_files), batch_size):
    batch_mask_files = mask_files[start:start + batch_size]
    current_bs = len(batch_mask_files)

    out = pipe(
        batch_size=current_bs,
        num_inference_steps=100,
        generator=generator,
    )

    for mask_name, im in zip(batch_mask_files, out.images):
        save_path = os.path.join(SAVE_DIR, mask_name)
        im.save(save_path)

print("Saved", len(mask_files), "samples to", SAVE_DIR)

Found 100 mask files, will generate 100 images.
Loading pipeline from: /content/drive/MyDrive/MSML612/ddim-breast_mri-256


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Saved 100 samples to /content/drive/MyDrive/MSML612/ddim-breast_mri-256/samples_from_test_masks
